# ChangeLog

1.  waiting to solution settle
2.  using only th acceleration data instead of the v and x

# Note
1.  try to remove the drop out in LSTM
2.  BE SURE data for chaotic case in Clean. (Training)
3.  Be sure for integration data is noise free.(Integrating)

In [1]:
import torch
import os
import tqdm
import model
import utils
import loss
import deeplearning
import torch.nn             as nn
import matplotlib.pyplot    as plt
import numpy                as np
import torch.optim          as optim

utils.set_seed(42)
device = 'cuda'

In [3]:
prediction_horizon      = utils.prediction_horizon
prediction_input_size   = utils.prediction_input_size
epochs                  = utils.num_epochs
_divition_factr         = utils.Noise_division_factor

model     = model.Encoder_Decoder().to(device)
criterion = loss.custum_loss(alpha=utils.alpha)#torch.nn.MSELoss()
optimizer = optim.SGD(
                        model.parameters(),
                        lr=5e-2,
                        momentum=0.9,
                        weight_decay=1e-4
                        )



optimizer_koopman = optim.SGD(
                        model.Koopman_operator.parameters(),
                        lr=5e-4,
                        # momentum=0.9,
                        # weight_decay=1e-4
                        )

In [4]:
npz_file_path = os.path.join("Duffing_Solution","datasets","gamma=0.37 t_span=(0, 50000) initial_conditions=[1.5, -1.5] step_frequency=010.npy")
loaded_data = utils.read_npz_file(npz_file_path)
data_tensor = torch.from_numpy(loaded_data).to(device=device).to(torch.float)

_Model_name = f"Gamma ={npz_file_path.split(' ')[0].split('=')[-1]} step_frequency={npz_file_path.split('.')[-2].split('=')[-1]} {prediction_input_size=} cu_loss={utils.Eigen} init=({npz_file_path.split('[')[-1].split(']')[0]}) noise_factor={_divition_factr}"

model, optimizer, report = deeplearning.train(
                                                data_tensor                 = data_tensor[0:100_000],
                                                prediction_input_size       = prediction_input_size,
                                                prediction_horizon          = utils.prediction_horizon,
                                                _divition_factr             = _divition_factr,
                                                
                                                model                       = model,
                                                model_name                  = _Model_name ,
                                                epochs                      = epochs,
                                                load_saved_model            = False,
                                                ckpt_save_freq              = 1 ,
                                                ckpt_save_path              = os.path.join("Saved" ,".Checkpoints"),
                                                ckpt_path                   = os.path.join("Saved" ,".Checkpoints","ckpt_Gamma =0.50 prediction_input_size=200 cu_loss=False init=(1, 0) noise_factor=1 gammas=0.37-0.29_epoch1.ckpt") ,
                                                report_path                 = "Saved" ,
                                                
                                                criterion                   = criterion,
                                                optimizer                   = optimizer,
                                                optimizer_koopman           = optimizer_koopman,
                                                lr_scheduler                = None,
                                                sleep_time                  = None,
                                                Validation_save_threshold   = None,
                                                device                      = 'cuda'    ,
                                                if_lstm                     = True
                                                )

  0%|          | 0/1 [08:15<?, ?it/s]


RuntimeError: Parent directory Saved\.Checkpoints does not exist.

In [ ]:
# # npz_file_path = "Duffing_Soulution\datasets\gamma=0.37 t_span=(0, 100000) initial_conditions=[0, 0.5].npy"
# # loaded_data = utils.read_npz_file(npz_file_path)
# # data_tensor = torch.from_numpy(loaded_data).to(device=device).to(torch.float)

# # model.load_state_dict(torch.load(os.path.join(directory_path)))
# # model.eval()


# _divition_factr = 3

# with torch.inference_mode():
#     Batch = 10
#     pred_hor = 1000
#     x = data_tensor[Batch*prediction_input_size:(Batch+1)*prediction_input_size]+((2*torch.rand(size=[prediction_input_size],device=device)-1)/(2*_divition_factr))
#     y = data_tensor[(Batch+1)*prediction_input_size:(Batch+1)*prediction_input_size+pred_hor]

#     prediction_list = torch.zeros(size=[pred_hor]).to(device)

#     decoder_hidden, decoder_cell = torch.zeros(size=[2,prediction_input_size],device=device), torch.zeros(size=[2,prediction_input_size],device=device)
#     for i in range(pred_hor):
#         # prediction = inception.forward(x)
#         prediction,(decoder_hidden, decoder_cell) = model.forward(x.unsqueeze(0),decoder_hidden, decoder_cell)
#         x =  torch.cat([x[1:],prediction],dim=0)
#         prediction_list[i] = prediction


# plt.figure(figsize=(20, 6),dpi=300)
# plt.plot((prediction_list[:]                                             ).detach().cpu().numpy() , linewidth=2,linestyle='dashed', label="Prediction")
# plt.plot((y[:]+(2*torch.rand(size=[pred_hor],device=device)-1)/(2*_divition_factr) ).detach().cpu().numpy() , alpha=0.3, label="Noisiy Ground Truth")
# plt.plot((y[:]                                                           ).detach().cpu().numpy() , alpha=0.5, label="Ground Truth")
# plt.legend()

In [ ]:
_prediction_list = []
_y               = []

# model.load_state_dict(torch.load(os.path.join('Saved\Gamma =0.2 step_frequency=010 prediction_input_size=400 cu_loss=False init=(1.5, -1.5) noise_factor=2.pt')))
# model.eval()

_divition_factr         = utils.Noise_division_factor

for adress     in [ 'Duffing_Soulution\datasets\gamma=0.37 t_span=(0, 50000) initial_conditions=[1.5, -1.5] step_frequency=010.npy',
                    # 'Duffing_Soulution\datasets\gamma=0.2 t_span=(0, 50000) initial_conditions=[0.5, 0.0] step_frequency=010.npy',
                    # 'Duffing_Soulution\datasets\gamma=0.2 t_span=(0, 50000) initial_conditions=[-0.5, 0.5] step_frequency=010.npy',
                        ]:
    for _divition_factr in [2,0.5,0.2]:
        loaded_data = utils.read_npz_file(adress)
        data_tensor = torch.from_numpy(loaded_data).to(device=device).to(torch.float)
        with torch.inference_mode():
            Batch = 10
            pred_hor = 500
            x = data_tensor[Batch*prediction_input_size:(Batch+1)*prediction_input_size]+((2*torch.rand(size=[prediction_input_size],device=device)-1)/(2*_divition_factr))
            y = data_tensor[(Batch+1)*prediction_input_size:(Batch+1)*prediction_input_size+pred_hor]

            prediction_list = torch.zeros(size=[pred_hor]).to(device)

            decoder_hidden, decoder_cell = torch.zeros(size=[2,prediction_input_size],device=device), torch.zeros(size=[2,prediction_input_size],device=device)
            for i in range(pred_hor):
                # prediction = inception.forward(x)
                prediction,(decoder_hidden, decoder_cell) = model.forward(x.unsqueeze(0),decoder_hidden, decoder_cell)
                x =  torch.cat([x[1:],prediction],dim=0)
                prediction_list[i] = prediction

        _prediction_list.append(prediction_list)
        _y.append(y)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

def plot_signals(_prediction_list,_y, descriptions,title,_divition_factr=_divition_factr):
    """
    Create three subplots with signals, legends, descriptions, and axis labels.
    Parameters:
    signals (list of arrays): List of signal arrays to be plotted.
    legends (list of str): List of legend labels for each signal.
    descriptions (list of str): List of descriptions for each signal.
    x_label (str): Label for the x-axis.
    y_label (str): Label for the y-axis.
    title (str): Title for the entire plot.
    """
    t = np.linspace(0, len(_y[0])/10, len(_y[0]))

    num_signals = len(_y)
    if num_signals != 3:
        raise ValueError("Exactly three signals are required.")
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 5),dpi=300)
    _divition_factr = [2,0.5,0.2]
    for i, ax in enumerate(axes):

        ax.plot(t,(_prediction_list[i]).detach().cpu().numpy(), label="Prediction",
                linestyle='dashed',
                color = 'b',
                linewidth=3)
        
        ax.plot(t,(_y[i]).detach().cpu().numpy(), label="Runge-Kutta solution",
                color='g',
                alpha=0.7,
                linewidth=3,)
        
        ax.plot(t,(_y[i]+(2*torch.rand(size=[pred_hor],device=device)-1)/(2*_divition_factr[i]) ).detach().cpu().numpy() , label="Noisy Runge-Kutta solution",
                color='#e69138',
                alpha=0.5)

        ax.set_xlabel('Time (s)',fontweight="bold", size=16)
        ax.set_ylabel('Acceleration (m/$s^2$)',fontweight="bold", size=16)
        ax.set_title(descriptions[i], size=16)
        ax.legend(loc='upper right')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        # ax.grid()
        # ax.set_xlim(xmin, xmax)
        ax.xaxis.set_major_locator(MultipleLocator(5))
        ax.yaxis.set_major_locator(MultipleLocator(0.5))

        ax.set_ylim(-3, 3) 
        ax.grid(which="major",alpha=0.6)
        ax.grid(which="minor",alpha=1)


    plt.suptitle(title,fontweight="bold", size=20)
    plt.tight_layout()

    plt.savefig(f"Images\Results\{title}.svg", format='svg')
    plt.show()


descriptions = ["a) Base input signal noise amplitude = [-0.25,0.25]", "b) noise amplitude = [-1,1]", "c) noise amplitude = [-2.5,2.5]"]
x_label = "Time"
y_label = "Amplitude"
title   = "Robustness against Noise gamma=0.37 [N]"

plot_signals(_prediction_list,_y,descriptions, title)


# Koopman

In [ ]:
def Koopman_Eigenvalue(
                       real_parts           : torch.tensor,
                       imaginary_parts      : torch.tensor,
                       magnitudes           : torch.tensor,
                       title                : str,
                       _index               : int = 256,
                       DPI                  : int = 400,
                       save_image           : bool= None,
                       neural_net_name      : str= None,
                       dataset_parameters   : str= None,
                       save_dir             : str = os.path.join('utils','Plot','Koopman_Eigenvalues')
                        )->None:
    
        real_parts_cpu      = real_parts.cpu().numpy()
        imaginary_parts_cpu = imaginary_parts.cpu().numpy()
        magnitudes_cpu      = magnitudes.cpu().numpy()
        
        fig , ax = plt.subplots(1,figsize=(15,10) , dpi=DPI)
        
        sc = ax.scatter(real_parts_cpu[:_index], imaginary_parts_cpu[:_index], c=magnitudes_cpu[:_index], cmap='viridis', marker='o')
        plt.colorbar(sc, label="Magnitude")
        
        ax.hlines(y=0,xmin=real_parts_cpu.min()     ,xmax=real_parts_cpu.max()      , color='black', linestyle='dashed')
        ax.vlines(x=0,ymin=imaginary_parts_cpu.min(),ymax=imaginary_parts_cpu.max() , color='black', linestyle='dashed')
        ax.set_xlabel('Real part ($\mu$)',fontweight="bold", size=16)
        ax.set_ylabel('Imaginary part ($\mu$)',fontweight="bold", size=16)
        ax.set_title(f"neural net = {neural_net_name} index={_index}",fontweight="bold", size=20)
        # ax.set_ylim(-1.25, 1.25) 
        # ax.set_xlim(-1.25, 1.25) 
        ax.grid(which="major",alpha=0.6)
        ax.grid(which="minor",alpha=1)
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        
        plt.grid(True)
        
        if save_image:
                plt.savefig(f"Images\Results\Koopamn {title}.svg", format='svg')
                
        plt.show()

In [ ]:
eigenvalues, eigenvectors = torch.linalg.eig(model.Koopman_operator.weight)

real_parts = torch.real(eigenvalues)
imaginary_parts = torch.imag(eigenvalues)
magnitudes = torch.abs(real_parts + 1j * imaginary_parts)

Koopman_Eigenvalue(real_parts.detach(),
                         imaginary_parts.detach(),
                         magnitudes.detach(),
                         "Koopman gamma=0.37",

                         _index         = 1600,
                        DPI             = 400,
                        save_image      = True,
                        neural_net_name = "CNN-RNN",
                        save_dir = '.')